In [1]:
import csv
import pandas as pd
import numpy as np
import os
import arff
import re
import codecs
path_to_dataset_folder = '..\\DATAsets\\DATA'
path_to_ready_csv = path_to_dataset_folder + '\\CSVs'
path_encoded_csv = path_to_dataset_folder + '\\CSVs_reverse_encoded'

In [10]:
df = pd.read_csv(path_to_dataset_folder + '/Target_feature_test_CSVs/OnlineNewsPopularity%kw_max_max.csv')
df.columns

Index(['timedelta', 'n_tokens_title', 'n_tokens_content', 'n_unique_tokens',
       'n_non_stop_words', 'n_non_stop_unique_tokens', 'num_hrefs',
       'num_self_hrefs', 'num_imgs', 'num_videos', 'average_token_length',
       'num_keywords', 'kw_min_min', 'kw_max_min', 'kw_avg_min', 'kw_min_max',
       'kw_avg_max', 'kw_min_avg', 'kw_max_avg', 'kw_avg_avg',
       'self_reference_min_shares', 'self_reference_max_shares',
       'self_reference_avg_sharess', 'is_weekend', 'LDA_00', 'LDA_01',
       'LDA_02', 'LDA_03', 'LDA_04', 'global_subjectivity',
       'global_sentiment_polarity', 'global_rate_positive_words',
       'global_rate_negative_words', 'rate_positive_words',
       'rate_negative_words', 'avg_positive_polarity', 'min_positive_polarity',
       'max_positive_polarity', 'avg_negative_polarity',
       'min_negative_polarity', 'max_negative_polarity', 'title_subjectivity',
       'title_sentiment_polarity', 'abs_title_subjectivity',
       'abs_title_sentiment_polarity', 

In [43]:
file = "OnlineNewsPopularity.csv"
df = pd.read_table(path_to_ready_csv + '\\'+ file,sep=',')
df.iloc[120:128,12:18]

,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world
120,0.0,1.0,0.0,0.0,0.0,0.0
121,0.0,0.0,0.0,0.0,1.0,0.0
122,0.0,0.0,0.0,0.0,1.0,0.0
123,0.0,0.0,0.0,0.0,0.0,1.0
124,0.0,0.0,0.0,0.0,0.0,1.0
125,0.0,0.0,0.0,1.0,0.0,0.0
126,0.0,1.0,0.0,0.0,0.0,0.0
127,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
types = df.dtypes
cardi = df.apply(pd.Series.nunique)
df2=pd.DataFrame({'Types': types,
                  'Cardinality': cardi})

In [53]:
df.iloc[107:115,df.shape[1]-1:df.shape[1]]

,type_of_channel
120,entertainment
121,tech
122,tech
123,world
124,world
125,socmed
126,entertainment
128,lifestyle


In [ ]:
#This is used to turn int-encoded features into str-encoded ones. This is desired
#because the str type is automatically recognised as categorical (enum) by H2O later.
#Set the variable 'features' to contain the int-encoded features to transform.
features = ["target_class"]
for target_feature in features:
    value_df = df[target_feature].value_counts(dropna=True).rename_axis('unique_values',axis=0).reset_index(name='value_counts',drop=False)
    value_df.sort_values(by='unique_values',inplace=True)
    value_df.reset_index(drop=True,inplace=True)
    lim = value_df.shape[0]
    replace_dict = {}
    for i in range(lim):
        replace_dict[ value_df.loc[i,'unique_values'] ] = 'Level_'+str(i)
    replace_dict = {target_feature : replace_dict}

    df.replace(to_replace = replace_dict, inplace=True, value=None)
df.head(10)

In [46]:
#Used to REVERSE ONE HOT ENCODING of the "invOneHotFeatures" CONSECUTIVE-features' vector,
#creating a new column with the name given in ~~~HERE_1~~~. A check is made if every feature
#belongs to exactly one category. If not, this is not usable, and that case needs to be dealt
#with manually. The values of the new column can be default (categoricals) if "autoValues" == 1,
#or manually defined IN THE CODE (~~~HERE_2~~~) for autoValues == 0.

invOneHotFeatures = list(range(12,18)) #Consecutive feature indices
#invOneHotFeatures = [2,8,12] #Separate features indices
autoValues = 0
df2=df.iloc[:,invOneHotFeatures]
ser = df2.sum(axis=1)
if len(ser[ser != 0]) == df.shape[0]:
    if ser[ser!=1].sum() == 0:
        reverse_enc = []
        for x in range(df.shape[0]):
            for feat in invOneHotFeatures:
                if df.iloc[x,feat] == 1:
                    if autoValues == 0:
                        #~~~HERE_2~~~
                        #Manual definition of values in the new column.
                        reverse_enc.append(df.columns.values[feat].split('_')[-1])
                    elif autoValues == 1:
                        #Automated value assignement as "value_0, value_1" etc.
                        reverse_enc.append('value_'+str(feat-min(invOneHotFeatures)))
                    break
        #~~~HERE_1~~~
        df=df.assign(type_of_channel=reverse_enc)
        df.drop(df.columns.values[invOneHotFeatures],axis=1,inplace=True)
        print(df.head(10))
    else:
        print('Not all features belong exclusively to one category!')
else:
    print(str(len(ser[ser == 0])) + ' out of '+ str(df.shape[0]) +' features ('+ str(round(len(ser[ser == 0])/df.shape[0]*10000)/100) +'%) belong to NO category!')

   timedelta  n_tokens_title  n_tokens_content  n_unique_tokens  \
0      731.0            12.0             219.0         0.663594   
1      731.0             9.0             255.0         0.604743   
2      731.0             9.0             211.0         0.575130   
3      731.0             9.0             531.0         0.503788   
4      731.0            13.0            1072.0         0.415646   
5      731.0            10.0             370.0         0.559889   
6      731.0             8.0             960.0         0.418163   
7      731.0            12.0             989.0         0.433574   
8      731.0            11.0              97.0         0.670103   
9      731.0            10.0             231.0         0.636364   

   n_non_stop_words  n_non_stop_unique_tokens  num_hrefs  num_self_hrefs  \
0               1.0                  0.815385        4.0             2.0   
1               1.0                  0.791946        3.0             1.0   
2               1.0               

In [45]:
#Used to drop problematic rows detected above.
df2=df.iloc[:,invOneHotFeatures]
ser = df2.sum(axis=1)
df.drop(ser[ser == 0].index,axis=0,inplace=True)

In [ ]:
df.to_csv(path_encoded_csv + '\\'+ file,index= False)